From https://planetarycomputer.microsoft.com/dataset/cil-gdpcir-cc0#Ensemble-example

In [1]:

import xarray as xr
import numpy as np
import pandas as pd
#from dask.diagnostics import ProgressBar
#from tqdm.auto import tqdm

import os
import re
import glob
import shutil
import zipfile
from pathlib import Path

import difflib
from scipy.spatial import KDTree

import matplotlib.pyplot as plt
import geopandas as gpd
import regionmask
import cartopy.crs as ccrs

from netCDF4 import Dataset

#from carbonplan import styles  # noqa: F401
#import intake
#import cmip6_downscaling
import dask

Load and organise data

Find lowest, median, and highest value model across all lat/long and across all time points

# Combine with grids for facilities

In [2]:

multiplier = 1 # no need for multiplier
years = range(2025, 2041) # final date is 1st Jan 2100
month_lengths = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] * len(years)
window_size = 5

facilities_with_lat_long = pd.read_csv(
    "/Users/rem76/Desktop/Climate_change_health/Data/facilities_with_lat_long_region.csv")

/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/3233272010.py:6: DtypeWarning: Columns (0: A402A_a3, 1: A402A_a4, 2: A417a, 3: A422b, 4: A425a, 5: A431, 6: AA400_c0815, 7: AA400_c0816, 8: B404a, 9: B407_a, 10: B408a, 11: C113A, 12: C404A16, 13: C404A17, 14: C404A__14, 15: C404C16, 16: C404C17, 17: C404E16, 18: C404E17, 19: C404Ea1, 20: C404Ea5, 21: C404Ea6, 22: C404Ea8, 23: C404Ea9, 24: C404F16, 25: C404F17, 26: C404G16, 27: C404G17, 28: C404Ga10, 29: C404Ga11, 30: C404Ga13, 31: C404Ga14, 32: C404Ga15, 33: C404Ga2, 34: C404Ga3, 35: C404Ga4, 36: C404Ga5, 37: C404Ga6, 38: C404Ga7, 39: C404Ga8, 40: D102_a, 41: D104B, 42: D108A3, 43: D108A4, 44: D108A5, 45: D108A6, 46: D108A__2, 47: D108A__3, 48: D108A__4, 49: D108A__5, 50: D108C3, 51: D108C4, 52: D108C5, 53: D108C6, 54: D108E3, 55: D108E4, 56: D108E5, 57: D108E6, 58: D111__0, 59: F101A, 60: J102a, 61: J103a, 62: M120a, 63: N106a, 64: Q103A, 65: interview__status) have mixed types. Specify dtype option on import or set low

In [3]:
def unzip_all_in_directory(directory):
    """
    Unzips all .zip files in the specified directory, extracting each into a separate folder.

    Parameters:
        directory (str): The path to the folder containing the .zip files.
    """
    for filename in os.listdir(directory):
        if filename.endswith('.zip'):
            file_path = os.path.join(directory, filename)
            extract_dir = os.path.join(directory, filename[:-4])
            os.makedirs(extract_dir, exist_ok=True)

            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_dir)
            except zipfile.BadZipFile:
                print(f"Skipped {filename}: not a valid zip file.")


def extract_nc_files_from_unzipped_folders(directory):
    """
    Searches for .nc files in the specified directory and all its subfolders,
    and copies them to the output directory, maintaining the folder structure.

    Parameters:
        directory (str): The path to the folder containing the unzipped folders.
    """
    output_directory = os.path.join(directory, 'nc_files')
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, _, files in os.walk(directory):
        # Skip the output directory to prevent recursive copying
        if root == output_directory:
            continue

        for filename in files:
            if filename.endswith('.nc'):
                source_file_path = os.path.join(root, filename)
                destination_file_path = os.path.join(output_directory, filename)

                # Only copy if the file does not already exist in the output directory
                if not os.path.exists(destination_file_path):
                    shutil.copy2(source_file_path, output_directory)

In [ ]:
base_dir = "/Users/rem76/Desktop/Climate_Change_Health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/"
scenarios = ["ssp245"]
window_size = 5

data_by_model_and_grid = {}

def calculate_cumulative_metrics(precip_data, window_size):
    """
    Calculate monthly totals and 5-day maximums for precipitation data.
    """
    # Monthly total
    monthly_total = np.sum(precip_data)

    # 5-day maximum using rolling window
    if len(precip_data) >= window_size:
        rolling_sums = np.cumsum(precip_data)
        rolling_sums[window_size:] -= rolling_sums[:-window_size]
        max_5_day = np.max(rolling_sums[window_size - 1:])
    else:
        max_5_day = np.sum(precip_data)  # Handle case where data is shorter than window size

    return monthly_total, max_5_day
for scenario in scenarios:
    print(f"Processing scenario: {scenario}")
    scenario_directory = os.path.join(base_dir, scenario)
    file_path_downscaled = f"/Users/rem76/Desktop/Climate_Change_Health/Data/Precipitation_data/Downscaled_CMIP6_data_CIL/"
    output_file = f"CIL_combined_{scenario}_2025_2041.nc"
    file_pattern = os.path.join(file_path_downscaled, "CIL_subset_ssp245_*.nc")
    data_all_models = xr.open_mfdataset(file_pattern, combine='nested', concat_dim="time")
    data_all_models.compute()
    files = sorted(Path(file_path_downscaled).glob(f"CIL_subset_{scenario}_*.nc"))

    pr_aggregated = data_all_models.mean(dim=["lat", "lon", "time"], skipna=True)
    min_model_object = pr_aggregated['pr'].idxmin(dim="model")
    min_model = min_model_object.values.item()

    overall_mean = pr_aggregated['pr'].mean(dim="model")
    abs_diff = abs(pr_aggregated['pr'] - overall_mean)
    mean_model_object = abs_diff.idxmin(dim="model")
    mean_model = mean_model_object.values.item()

    max_model_object = pr_aggregated['pr'].idxmax(dim="model")
    max_model = max_model_object.values.item()

    models_of_interest = [min_model, mean_model, max_model]
    print("Models of interest", models_of_interest)

    #Initialize cumulative storage for models of interest
    cumulative_weather_dfs = {
        model: {"monthly": pd.DataFrame(), "window": pd.DataFrame()}
        for model in models_of_interest
    }

    for file_path in files:
        print(f"Processing file: {file_path.name}")
        data_all_models = xr.open_dataset(file_path)

        for model in models_of_interest:
            if model not in data_all_models["model"].values:
                print(f"Model {model} not found in file {file_path.name}, skipping.")
                continue

            print(f"Processing model: {model}")
            data_per_model = data_all_models.sel(model=model)

            # Prepare grid data structure
            lat_data = data_per_model.variables['lat'][:]
            lon_data = data_per_model.variables['lon'][:]
            lon_grid, lat_grid = np.meshgrid(lon_data, lat_data)
            centroids = np.column_stack((lat_grid.ravel(), lon_grid.ravel()))

            grid_precip_map = {}

            for year in np.unique(data_per_model['time.year']):
                for month in range(1, 13):  # 1 to 12 for each month
                    print(f"Processing year {year}, month {month}")

                    # Extract precipitation data for this month of this year
                    month_data = data_per_model.sel(
                        time=data_per_model.time.dt.year == year
                    ).sel(time=data_per_model.time.dt.month == month)

                    # Skip if no data
                    if month_data.time.size == 0:
                        continue

                    # Get daily precipitation values for grids
                    for grid, (i, j) in enumerate(np.ndindex(len(lat_data), len(lon_data))):
                        precip_data_for_grid = month_data.isel(lat=i, lon=j).pr.values

                        if grid not in grid_precip_map:
                            grid_precip_map[grid] = {"monthly": {}, "window": {}}

                        # Calculate metrics
                        monthly, window = calculate_cumulative_metrics(precip_data_for_grid, window_size)

                        grid_precip_map[grid]["monthly"][(year, month)] = monthly
                        grid_precip_map[grid]["window"][(year, month)] = window

            # Map facilities to grids and assign metrics
            kd_tree = KDTree(centroids)

            for facility in facilities_with_lat_long['Fname'].unique():
                lat = facilities_with_lat_long.loc[facilities_with_lat_long['Fname'] == facility, "A109__Latitude"].iloc[0]
                lon = facilities_with_lat_long.loc[facilities_with_lat_long['Fname'] == facility, "A109__Longitude"].iloc[0]
                if not np.isnan(lat) and not np.isnan(lon):
                    facility_location = np.array([[lat, lon]])
                    dist, closest_grid_index = kd_tree.query(facility_location)
                    closest_grid_index = closest_grid_index[0]

                    for (year, month), metrics in grid_precip_map[closest_grid_index]["monthly"].items():
                        cumulative_weather_dfs[model]["monthly"].loc[f"{year}-{month}", facility] = metrics
                    for (year, month), metrics in grid_precip_map[closest_grid_index]["window"].items():
                        cumulative_weather_dfs[model]["window"].loc[f"{year}-{month}", facility] = metrics
    model_categories = {
    min_model: "lowest",
    mean_model: "mean",
    max_model: "highest"
}

# Save cumulative results
    for model, weather_dfs in cumulative_weather_dfs.items():
        category = model_categories[model]  # Get the category for the model
        for metric_type, df in weather_dfs.items():
            # Use the category in the output file name
            output_file = Path(scenario_directory) / f"{category}_{metric_type}_prediction_weather_by_facility.csv"
            df.to_csv(output_file, index=True)
            print(f"Saved {metric_type} data for {category} model to {output_file}")



Processing scenario: ssp245
Models of interest ['HadGEM3-GC31-LL', 'UKESM1-0-LL', 'INM-CM5-0']
Processing file: CIL_subset_ssp245_2024.nc
Processing model: HadGEM3-GC31-LL
Processing year 2024, month 1
Processing year 2024, month 2
Processing year 2024, month 3
Processing year 2024, month 4
Processing year 2024, month 5
Processing year 2024, month 6
Processing year 2024, month 7
Processing year 2024, month 8
Processing year 2024, month 9
Processing year 2024, month 10
Processing year 2024, month 11
Processing year 2024, month 12


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["monthly"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["window"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFr

Processing model: UKESM1-0-LL
Processing year 2024, month 1
Processing year 2024, month 2
Processing year 2024, month 3
Processing year 2024, month 4
Processing year 2024, month 5
Processing year 2024, month 6
Processing year 2024, month 7
Processing year 2024, month 8
Processing year 2024, month 9
Processing year 2024, month 10
Processing year 2024, month 11
Processing year 2024, month 12


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["monthly"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["window"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFr

Processing model: INM-CM5-0
Processing year 2024, month 1
Processing year 2024, month 2
Processing year 2024, month 3
Processing year 2024, month 4
Processing year 2024, month 5
Processing year 2024, month 6
Processing year 2024, month 7
Processing year 2024, month 8
Processing year 2024, month 9
Processing year 2024, month 10
Processing year 2024, month 11
Processing year 2024, month 12


/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["monthly"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cumulative_weather_dfs[model]["window"].loc[f"{year}-{month}", facility] = metrics
/var/folders/1z/j8w4v5lj4k580xt42fkwh7dw0000gn/T/ipykernel_69502/793788602.py:112: PerformanceWarning: DataFr

Processing file: CIL_subset_ssp245_2025.nc
Processing model: HadGEM3-GC31-LL
Processing year 2025, month 1
Processing year 2025, month 2
Processing year 2025, month 3
Processing year 2025, month 4
Processing year 2025, month 5
Processing year 2025, month 6
Processing year 2025, month 7
Processing year 2025, month 8
Processing year 2025, month 9
Processing year 2025, month 10
Processing year 2025, month 11
Processing year 2025, month 12
Processing model: UKESM1-0-LL
Processing year 2025, month 1
Processing year 2025, month 2
Processing year 2025, month 3
Processing year 2025, month 4
Processing year 2025, month 5
Processing year 2025, month 6
Processing year 2025, month 7
Processing year 2025, month 8
Processing year 2025, month 9
Processing year 2025, month 10
Processing year 2025, month 11
Processing year 2025, month 12
Processing model: INM-CM5-0
Processing year 2025, month 1
Processing year 2025, month 2
Processing year 2025, month 3
Processing year 2025, month 4
Processing year 202

In [6]:
facilities_with_lat_long

,interview__id,Fcode,Fname,A102b,A103,newfacility,A110,Zonename,A112_aa,Dist,...,X113,X114,comment,has__errors,interview__key,interview__status,missing_C113,samplefacility,ssSys_IRnd,sssys_irnd
0,00486c9ed0b94db1a482f2a6a6ec0768,575,Mary Mount Clinic,mary mount,NaN,No,NORTH,Northern,NaN,Mzimba North,...,No,No,NaN,0,72-83-70-37,NaN,No,MOH,NaN,0.329296
1,00ada6f69b6343b1b1bbbde173e35542,503,Edingeni Health Centre (COM),Edingeni,MCM/MOH/HC/32,No,NORTH,Northern,NaN,Mzimba South,...,"Yes, observed",No,NaN,0,39-21-43-13,NaN,No,COM,NaN,0.971593
2,00bb7dfdbfd34f23a73a551b8d2c8cfe,128,KAVUKUKU,Kavukuku,NaN,No,NORTH,Northern,Nthalire,Chitipa,...,"Yes, observed",No,NaN,0,59-53-97-79,NaN,No,MOH,NaN,0.561124
3,00d18982c2d1438db94c019d5549fa78,8338,Healing Hearts Private Clinic,Healing Hearts,MCM/C/384,Yes,SOUTH,South West,NaN,Blanytyre,...,No,No,NaN,0,15-22-61-24,NaN,No,MOH,NaN,0.635793
4,00da0050e6054e928eeb24a87a52623a,1581,Lilongwe City Assembly Chinsap,cha city pa ma kesha,NaN,No,CENTER,Central West,NaN,Lilongwe,...,No,"Yes, not observed",NaN,0,80-13-23-05,NaN,No,MOH,NaN,0.157339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,ff0af2b6712d4f53aab84331c64bfc89,1533,Nambuma Health Centre,Nambuma,MCM/CHAM/RH/44,No,CENTER,Central West,NaN,Lilongwe,...,"Yes, observed","Yes, observed",NaN,0,91-26-45-02,NaN,No,MOH,NaN,0.503917
1102,ff66df6436d0401e8da4ab62d6ec2a86,3402,Kalemba Community Hospital,Kalemba,HC/28,No,SOUTH,South West,NaN,Nsanje,...,"Yes, observed","Yes, not observed",Nil,0,43-03-66-34,NaN,No,MOH,NaN,0.491651
1103,ff69fd8e8828470b82358c8abd201253,5049,Central East African Railways,Railways Private Clinic,MCM/C/021,No,SOUTH,South West,NaN,Blanytyre,...,No,No,NaN,0,45-96-56-91,NaN,No,MOH,NaN,0.442132
1104,ff6e33f51f98483aa2fca37d75031c7c,5032,"Thejopal, Rita S.",Dr Kayange,NaN,No,SOUTH,South West,NaN,Blanytyre,...,NaN,NaN,NaN,0,02-34-06-66,NaN,No,MOH,NaN,0.023951
